In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import datasets

from models import ControlledAutoEncoder

from tqdm.auto import tqdm, trange
import numpy as np
from PIL import Image

from utils import display_img, output_to_pil

In [2]:
mnist_trainset = datasets.MNIST(root="../data/", train=True,
                                download=True,
                                transform=torchvision.transforms.Compose([
                                    torchvision.transforms.ToTensor(),
                                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                                ]))

mnist_valid = datasets.MNIST(root="../data/", train=False,
                             download=True,
                             transform=torchvision.transforms.Compose([
                                 torchvision.transforms.ToTensor(),
                                 torchvision.transforms.Normalize((0.1307,), (0.3081,))
                             ]))

In [3]:
model = ControlledAutoEncoder().to("cuda")
pixel_loss_fn = nn.MSELoss().to("cuda")
controls_loss_fn = nn.NLLLoss().to("cuda")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
dataloader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, shuffle=True)
valid_loader = torch.utils.data.DataLoader(mnist_valid, batch_size=1, shuffle=False)

In [6]:
for epoch in range(10):

    train_loss = 0
    valid_loss = 0

    model.train()
    for images, labels in tqdm(dataloader, leave=False):
        optimizer.zero_grad()
        controls, output = model(images.to("cuda"))
        loss = pixel_loss_fn(output, images.to("cuda")) + controls_loss_fn(torch.log(controls + 1e-20), labels.to("cuda"))
        loss.backward()
        optimizer.step()
        train_loss += loss.item() / len(dataloader)

    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(valid_loader, leave=False):
            controls, output = model(images.to("cuda"))
            loss = pixel_loss_fn(output, images.to("cuda")) + controls_loss_fn(torch.log(controls + 1e-20), labels.to("cuda"))
            valid_loss += loss.item() / len(valid_loader)

    print(f"Epoch {epoch+1}: train_loss {train_loss}, valid_loss {valid_loss}")

Epoch 1: train_loss 0.5270159248350017, valid_loss 0.5004888501297675
Epoch 2: train_loss 0.49053422118554973, valid_loss 0.4865106708683086
Epoch 3: train_loss 0.47187075241288157, valid_loss 0.48184355372264803
Epoch 4: train_loss 0.4615588052186377, valid_loss 0.4774765100687739
Epoch 5: train_loss 0.4564324872834346, valid_loss 0.47889389554895173
Epoch 6: train_loss 0.45120251210513646, valid_loss 0.46516422527618506
Epoch 7: train_loss 0.4470279268872759, valid_loss 0.46584136868342835
Epoch 8: train_loss 0.44487881666815865, valid_loss 0.46726217034682505
Epoch 9: train_loss 0.43991272980724566, valid_loss 0.46495159386098506
Epoch 10: train_loss 0.4373976358218486, valid_loss 0.4665291009332988


In [33]:
for i in range(10):
    digit = F.one_hot(torch.tensor(i), num_classes=10)
    for _ in range(10):
        model.eval()
        with torch.no_grad():
            output = model.decoder(torch.rand(1, 1).cuda()*10-5, digit.unsqueeze(0).cuda())
        display_img(output_to_pil(output))